In [13]:
pip install chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [3]:
from bs4 import BeautifulSoup
import os
import requests  # Make sure you import requests
from urllib.parse import urljoin  # Add this import statement

def get_program_links(catalog_url):
    """
    Extracts all program links from the given catalog page URL.
    """
    try:
        response = requests.get(catalog_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            program_links = []
            # Find all <a> tags inside <ul class="program-list">
            for link in soup.select('ul.program-list a'):
                href = link['href']
                program_links.append(urljoin(catalog_url, href))  # Convert relative URL to absolute
            return program_links
        else:
            print(f"Failed to retrieve the page: {catalog_url}")
            return []
    except Exception as e:
        print(f"Error fetching page {catalog_url}: {e}")
        return []


# URL of the course catalog page
catalog_url = "https://catalog.newhaven.edu/content.php?catoid=31&navoid=2062"

# Get program links from the catalog page
program_links = get_program_links(catalog_url)
print(f"Found {len(program_links)} program links.")

# Now, for each program page, visit the link and look for PDF download links
for program_url in program_links:
    # Now you can parse each program URL for PDF links (or other info)
    print(f"Processing program page: {program_url}")

Found 158 program links.
Processing program page: https://catalog.newhaven.edu/preview_program.php?catoid=31&poid=8789&returnto=2062
Processing program page: https://catalog.newhaven.edu/preview_program.php?catoid=31&poid=8855&returnto=2062
Processing program page: https://catalog.newhaven.edu/preview_program.php?catoid=31&poid=8854&returnto=2062
Processing program page: https://catalog.newhaven.edu/preview_program.php?catoid=31&poid=8861&returnto=2062
Processing program page: https://catalog.newhaven.edu/preview_program.php?catoid=31&poid=8825&returnto=2062
Processing program page: https://catalog.newhaven.edu/preview_program.php?catoid=31&poid=8807&returnto=2062
Processing program page: https://catalog.newhaven.edu/preview_program.php?catoid=31&poid=8806&returnto=2062
Processing program page: https://catalog.newhaven.edu/preview_program.php?catoid=31&poid=8805&returnto=2062
Processing program page: https://catalog.newhaven.edu/preview_program.php?catoid=31&poid=8886&returnto=2062
Pro

In [4]:
def get_program_details(program_url):
    """
    Fetches and parses the program details from a given program page URL.
    """
    try:
        response = requests.get(program_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Target the specific <table class="table_default">
            details_table = soup.find('table', class_='table_default')
            if details_table:
                return details_table.text.strip()  # Get the text content of the table
            else:
                return "Details not found."
        else:
            return f"Failed to retrieve the program page: {program_url}"
    except Exception as e:
        return f"Error fetching program details {program_url}: {e}"

# Example usage:
for program_url in program_links:
    details = get_program_details(program_url)
    print(f"Details for {program_url}: {details}")


Details for https://catalog.newhaven.edu/preview_program.php?catoid=31&poid=8789&returnto=2062: University of New Haven








﻿ Section

                    Academic Catalogs 










2024-2025 Undergraduate Catalog


2024-2025 Graduate Catalog


﻿Archived Catalogs


2023-2024 Graduate Catalog


2023-2024 Undergraduate Catalog


2022-2023 Graduate Catalog


2022-2023 Undergraduate Catalog


2021-2022 Graduate Catalog


2021-2022 Undergraduate Catalog


2020-2021 Undergraduate Catalog


2020-2021 Graduate Catalog


2019-2020 Undergraduate Catalog


2019-2020 Graduate Catalog


2018-2019 Undergraduate Catalog


2018-2019 Graduate Catalog


2017-2018 Undergraduate Catalog


2017-2018 Graduate Catalog


2016-2017 Undergraduate Catalog


2016-2017 Graduate Catalog


2015-2016 Undergraduate Catalog


2015-2016 Graduate Catalog


2014-2015 Undergraduate Catalog


2013-2015 Graduate Catalog


2012-2014 Undergraduate Catalog


2011-2013 Graduate Catalog


2010-2012 Undergraduate Catalog


2

In [6]:
import re
import os

def clean_text(text):
    unwanted_phrases = [
        "University of New Haven", "Academic Catalogs", "Archived Catalogs", 
        "Select a Catalog", "HELP", "Back to Top", "Global Search",
        "Search Keyword Field", "Catalog Navigation", "President’s Message",
        "Financial Aid", "Graduate Tuition and Fees", "Campus Map", 
        "Board of Governors", "Administration", "Faculty",
        "Section", "300 Boston Post Rd", "West Haven, CT 06516", "1-800-342-5864", 
        "Online Privacy Statement", "Send Us Feedback", "Feb 13, 2025",
        "D.H.Sc.", "Print Degree Planner (opens a new window)", 
        "Print-Friendly Page (opens a new window)","Return to:", "Search Choose Search Location Select an option Courses Programs Hierarchy Items Other Content Entire Catalog Whole Word/Phrase Advanced Search Catalog Home"
    ]

    lines = text.split('\n')
    filtered_lines = []
    for line in lines:
        if any(phrase in line for phrase in unwanted_phrases):
            continue
        if re.match(r"\d{4}-\d{4} (Graduate|Undergraduate) Catalog", line.strip()):
            continue
        if line.strip():
            filtered_lines.append(line.strip())
    cleaned_text = ' '.join(filtered_lines)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return cleaned_text

def save_cleaned_text(details, filename, all_text_file, folder_path):
    cleaned_details = clean_text(details)
    print(cleaned_details)  # Continue to print the cleaned details

    # Save to individual files
    with open(os.path.join(folder_path, filename), 'w') as file:
        file.write(cleaned_details + '\n')

    # Save to a single file
    with open(all_text_file, 'a') as file:
        file.write(cleaned_details + '\n')

# Create a folder for individual text files
folder_name = 'CleanedTexts'
os.makedirs(folder_name, exist_ok=True)

# Define the path for the single text file
single_text_file = 'combined_cleaned_texts.txt'
# Clear the content of the combined file if it already exists
open(single_text_file, 'w').close()

# Example usage:
for i, program_url in enumerate(program_links):
    details = get_program_details(program_url)
    file_name = f'cleaned_text_{i}.txt'
    save_cleaned_text(details, file_name, single_text_file, folder_name)


﻿ Feb 23, 2025 Criminal Justice, Ph.D. Catalog Search Choose Search Location Select an option Courses Programs Hierarchy Items Other Content Entire Catalog Whole Word/Phrase Advanced Search Catalog Home The University Graduate Programs Academic Policies Policies & Procedures for Ph.D. Degree Academic and Student Services College of Arts and Sciences Pompea College of Business Tagliatela College of Engineering Henry C. Lee College of Criminal Justice and Forensic Sciences School of Health Sciences University Interdisciplinary Collaborative Programs Pending Regulatory Approval Course Descriptions Academic Calendar
﻿ Feb 23, 2025 Engineering and Applied Science, Ph.D. The research-based Doctorate of Philosophy (Ph.D.) in Engineering and Applied Science prepares students for positions in academia, industry, or consulting in a specific engineering or applied science discipline with coursework and research tailored to the student's particular interests and professional goals. Students identi

In [26]:
import os
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from sentence_transformers import SentenceTransformer
import tiktoken
import json

# Set up folder path
folder_path = "CleanedTexts"  # Change to your actual folder

# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize ChromaDB client & collection
chroma_client = chromadb.PersistentClient(path="./vector_db")
collection = chroma_client.get_or_create_collection(name="course_embeddings",
    embedding_function=SentenceTransformerEmbeddingFunction("all-MiniLM-L6-v2"))

# Function to split text into chunks
def split_text(text, chunk_size=500):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i+chunk_size])
        token_count = len(tokenizer.encode(chunk))
        chunks.append({"text": chunk, "tokens": token_count})
    
    return chunks

# Process all text files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read().replace("\ufeff", "")
            
            # Split text into chunks
            chunks = split_text(text)
            
            # Store chunks in ChromaDB with embeddings
            for idx, chunk in enumerate(chunks):
                embedding = embedding_model.encode(chunk["text"]).tolist()
                collection.add(
                    ids=[f"{filename}_{idx}"],
                    documents=[chunk["text"]],
                    embeddings=[embedding]
                )

print("✅ Data successfully stored in ChromaDB!")

# Function to search for relevant text in ChromaDB
def search_in_chromadb(query, top_k=3):
    query_embedding = embedding_model.encode(query).tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)

    for i, doc in enumerate(results["documents"][0]):
        print(f"\n Match {i+1}:\n{doc}")

# Example query for testing
query_text = "What courses are required for the datascience program?"
search_in_chromadb(query_text)


✅ Data successfully stored in ChromaDB!

 Match 1:
Feb 18, 2025 Data Science, M.S. STEM Designation: This program is STEM (science, technology, engineering, and math)-designated by the Department of Homeland Security. For more information, please see https://www.newhaven.edu/admissions/stem-designated-programs.php The Master of Science in Data Science (MSDS) program is designed to prepare students for opportunities and challenges in the field of data science. The program offers a strong foundation in data analysis, machine learning, artificial intelligence, and statistical modeling. Specialized focus areas include big data, deep learning, computer vision, and natural language processing with opportunities to gain subject matter expertise in a wide range of cross-disciplinary fields such as security, business analytics, and health care analytics. To earn the MSDS degree, a student must complete a minimum of 33 credits. One of these courses (3 credits) is considered a prerequisite prepar

In [28]:
# Example query for testing
query_text = "How many credits are required to complete master's course in Sport Management?"
search_in_chromadb(query_text)


 Match 1:
Feb 18, 2025 A total of 30-36 credits are required for completion of the master of science in Sport Management (depending on the choice of concentration). The program is made up of a Sport Management core that is common to each of the concentrations (consisting of six Sport Management courses) and a set of concentration courses (consisting of between four and six courses tailored to that particular segment of the Sport Management discipline). Sport Management Core (18 credits)SMGT 6600 - Sport Business ToolkitSMGT 6610 - Strategic Sport ManagementSMGT 6611 - Sport Industry Marketing, Promotion and Public RelationsSMGT 6612 - Sports LawSMGT 6617 - Managing Sport FinancesSMGT 6694 - Internship I (Capstone Course)Concentration requirements (6 credits)SMGT 6618 - Major Policy Issues in Intercollegiate AthleticsSMGT 6621 - Applied Collegiate Fitness and AthleticsPlus two of the following electives (6 credits)SMGT 6613 - Sports Facility ManagementSMGT 6620 - Professional Sport Man

In [41]:
import os
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from sentence_transformers import SentenceTransformer
import tiktoken
import re

# Set up folder path
folder_path = "CleanedTexts"

# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./vector_db")
collection = chroma_client.get_or_create_collection(name="course_embeddings",
    embedding_function=SentenceTransformerEmbeddingFunction("all-MiniLM-L6-v2"))

# Function to split text into smaller chunks (200-300 words)
def split_text(text, chunk_size=300):
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    
    return chunks

# Process all text files
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read().replace("\ufeff", "")
            
            # Split text into chunks
            chunks = split_text(text)
            
            # Store chunks in ChromaDB with embeddings
            for idx, chunk in enumerate(chunks):
                collection.add(
                    ids=[f"{filename}_{idx}"],
                    documents=[chunk]
                )

print("✅ Data successfully stored in ChromaDB!")


Add of existing embedding ID: cleaned_text_24.txt_0
Insert of existing embedding ID: cleaned_text_24.txt_0
Add of existing embedding ID: cleaned_text_24.txt_1
Insert of existing embedding ID: cleaned_text_24.txt_1
Add of existing embedding ID: cleaned_text_30.txt_0
Insert of existing embedding ID: cleaned_text_30.txt_0
Add of existing embedding ID: cleaned_text_18.txt_0
Insert of existing embedding ID: cleaned_text_18.txt_0
Add of existing embedding ID: cleaned_text_18.txt_1
Insert of existing embedding ID: cleaned_text_18.txt_1
Add of existing embedding ID: cleaned_text_1.txt_0
Insert of existing embedding ID: cleaned_text_1.txt_0
Add of existing embedding ID: cleaned_text_1.txt_1
Insert of existing embedding ID: cleaned_text_1.txt_1
Add of existing embedding ID: cleaned_text_1.txt_2
Insert of existing embedding ID: cleaned_text_1.txt_2
Add of existing embedding ID: cleaned_text_0.txt_0
Insert of existing embedding ID: cleaned_text_0.txt_0
Add of existing embedding ID: cleaned_text_19

✅ Data successfully stored in ChromaDB!


In [42]:
import re

def search_in_chromadb(query, top_k=3):
    query_embedding = embedding_model.encode(query).tolist()
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)

    # Correct way to handle potential list of documents:
    best_text = ""  # Initialize to empty string
    if results["documents"]:
      if isinstance(results["documents"][0], list): #Check if it is a list
        best_text = "\n".join(results["documents"][0]) #Join the list if it is a list
      else:
        best_text = results["documents"][0] #If not a list, then take the first element
    return best_text

def extract_program_details(text):
    """
    Extracts structured details like credits, core courses, and electives.
    """
    if not text: # Handle empty text
        return {"Program": "Not found", "Credits Required": "Not found", "Core Courses": []}

    try: # Add a try-except block to handle potential errors.
      program_name = text.split("\n")[0]  # First line is usually the program name
    except IndexError: # If the text has no newline
      program_name = text

    credit_match = re.findall(r"(\d{1,2})\s*(?:credit|credits|credit hours)", text, re.IGNORECASE)
    credits = credit_match[0] if credit_match else "Not specified"

    core_courses = re.findall(r"([A-Z]{3,4}\s\d{4} - .+?)\n", text)

    return {
        "Program": program_name,
        "Credits Required": credits,
        "Core Courses": core_courses[:5]  # Limit to 5 for clarity
    }

# ... (rest of your code)
# Test query
query_text = "What courses are required for the Data Science program?"
retrieved_text = search_in_chromadb(query_text)

# Extract structured details
course_info = extract_program_details(retrieved_text)
print("\n🎯 Extracted Course Information:", course_info)



🎯 Extracted Course Information: {'Program': 'PlanA recommended course plan (30 credit hours) for students who are NOT required to take DSCI 6602 - Introduction to Programming for Data Science: First Semester DSCI 6001 - Math for Data Scientists DSCI 6002 - Intro to Data Science DSCI 6612 - Intro to Artificial Intelligence Second Semester DSCI 6003 - Machine Learning DSCI 6007 - Distributed & Scalable Data Engineering Elective/Special Topic Course Third Semester DSCI 6004 - Natural Language Processing DSCI 6011 - Deep Learning Elective/Special Topic Course Fourth Semester DSCI 6010 - Data Science Internship or Capstone Project A recommended course plan (33 credit hours) for students who are required to take DSCI 6602 - Introduction to Programming for Data Science: First Semester DSCI 6602 - Introduction to Programming for Data Science DSCI 6001 - Math for Data Scientists DSCI 6002 - Intro to Data Science Second Semester DSCI 6003 - Machine Learning DSCI 6612 - Intro to Artificial Intel

In [62]:
query_text = "How many credits are required in data science program?"
retrieved_text = search_in_chromadb(query_text)

# Extract structured details
course_info = extract_program_details(retrieved_text)
print("\n🎯 Extracted Course Information:", course_info)


🎯 Extracted Course Information: {'Program': 'Feb 18, 2025 Data Science, M.S. STEM Designation: This program is STEM (science, technology, engineering, and math)-designated by the Department of Homeland Security. For more information, please see https://www.newhaven.edu/admissions/stem-designated-programs.php The Master of Science in Data Science (MSDS) program is designed to prepare students for opportunities and challenges in the field of data science. The program offers a strong foundation in data analysis, machine learning, artificial intelligence, and statistical modeling. Specialized focus areas include big data, deep learning, computer vision, and natural language processing with opportunities to gain subject matter expertise in a wide range of cross-disciplinary fields such as security, business analytics, and health care analytics. To earn the MSDS degree, a student must complete a minimum of 33 credits. One of these courses (3 credits) is considered a prerequisite preparatory